In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,54,2024-02-13,4,120.433962,97.671698,0.569509,10.628302,24.445283,0.198094,122.275472,...,10.536538,25.351923,0.177769,116.548077,0,2,1,2,2,2024
1,54,2024-02-13,1,120.698113,98.849057,0.576000,10.515094,20.684906,0.207849,121.196226,...,12.379245,25.894340,0.229434,113.920755,0,2,1,2,21,2024
2,54,2024-02-13,-1,110.377358,96.083019,0.533698,11.198113,22.324528,0.223151,114.669811,...,10.920370,21.992593,0.225444,121.007407,0,2,1,2,16,2024
3,53,2024-02-13,-1,118.596154,99.300000,0.565250,11.290385,23.086538,0.177096,118.186538,...,12.600000,24.626415,0.235302,119.679245,0,2,1,2,26,2024
4,54,2024-02-13,2,113.566038,96.713208,0.567226,12.932075,22.986792,0.220057,116.911321,...,12.401923,26.088462,0.197231,110.494231,0,2,1,1,18,2024
5,53,2024-02-13,-1,113.153846,99.628846,0.538692,13.098077,25.223077,0.197750,112.959615,...,11.935185,19.738889,0.218333,115.275926,0,2,1,3,9,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'FCNN':
        fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
        fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)
        
        model_file = fcnns.iloc[0]['File']
        
        model = torch.load('Models/'+model_file)
        scaler = joblib.load('Models/Scalers/FCNN.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'FCNN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-13,BOS,BRK,1.266667,3.950000,1
1,2024-02-13,OKC,ORL,1.704225,2.200000,1
2,2024-02-13,MIA,MIL,3.600000,1.307692,0
3,2024-02-13,SAC,PHO,2.700000,1.487805,0
4,2024-02-13,MIN,POR,1.277778,3.850000,1
5,2024-02-13,DET,LAL,4.900000,1.192308,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-13,BOS,BRK,1.266667,3.950000,1
1,2024-02-13,OKC,ORL,1.704225,2.200000,1
2,2024-02-13,MIA,MIL,3.600000,1.307692,0
3,2024-02-13,SAC,PHO,2.700000,1.487805,0
4,2024-02-13,MIN,POR,1.277778,3.850000,1
5,2024-02-13,DET,LAL,4.900000,1.192308,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-13,BOS,BRK,1.266667,3.950000,1
1,2024-02-13,OKC,ORL,1.704225,2.200000,1
2,2024-02-13,MIA,MIL,3.600000,1.307692,0
3,2024-02-13,SAC,PHO,2.700000,1.487805,0
4,2024-02-13,MIN,POR,1.277778,3.850000,1
5,2024-02-13,DET,LAL,4.900000,1.192308,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-13,BOS,BRK,1.266667,3.950000,1
1,2024-02-13,OKC,ORL,1.704225,2.200000,1
2,2024-02-13,MIA,MIL,3.600000,1.307692,0
3,2024-02-13,SAC,PHO,2.700000,1.487805,0
4,2024-02-13,MIN,POR,1.277778,3.850000,1
5,2024-02-13,DET,LAL,4.900000,1.192308,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-13,BOS,BRK,1.266667,3.950000,1.0
1,2024-02-13,OKC,ORL,1.704225,2.200000,1.0
2,2024-02-13,MIA,MIL,3.600000,1.307692,0.0
3,2024-02-13,SAC,PHO,2.700000,1.487805,0.0
4,2024-02-13,MIN,POR,1.277778,3.850000,1.0
5,2024-02-13,DET,LAL,4.900000,1.192308,0.0
